# ANÁLISIS EXPLORATORIO DE DATOS

## 1. Importar Librerias

In [110]:
# Librerias

import pandas as pd
import os
from pathlib import Path
import re
from tqdm import tqdm

In [104]:
!pip install xlrd

## 2. Traer Datos

### 2.1. Traer Datos de Defunciones de 1979 - 1991

In [24]:
# Ruta de los archivos
ruta_muertes = Path("data/raw/Muertes")

# Verificar si la carpeta existe
if not ruta_muertes.exists():
    print(f"❌ La carpeta {ruta_muertes} no existe")
else:
    # Listar todos los archivos .txt en la carpeta
    archivos = list(ruta_muertes.glob("Defun*.txt"))
    print(f"📁 Archivos encontrados: {[arch.name for arch in archivos]}")
    
    # Filtrar por años 1979-1991
    archivos_filtrados = [arch for arch in archivos if 1979 <= int(arch.stem[-4:]) <= 1991]
    
    dataframes = {}
    
    for archivo in archivos_filtrados:
        try:
            # Extraer año del nombre del archivo
            año = archivo.stem[-4:]
            
            # Leer archivo (prueba diferentes encodings si es necesario)
            df = pd.read_csv(
                archivo,
                sep='\t',        
                encoding='utf-8', 
                low_memory=False,
                dtype=str
            )
            
            dataframes[f'defun_{año}'] = df
            print(f"✅ {archivo.name}: {len(df)} filas, {len(df.columns)} columnas")
            
        except Exception as e:
            print(f"❌ Error con {archivo.name}: {e}")

📁 Archivos encontrados: ['Defun1979.txt', 'Defun1980.txt', 'Defun1981.txt', 'Defun1982.txt', 'Defun1983.txt', 'Defun1984.txt', 'Defun1985.txt', 'Defun1986.txt', 'Defun1987.txt', 'Defun1988.txt', 'Defun1989.txt', 'Defun1990.txt', 'Defun1991.txt', 'Defun1992.txt', 'Defun1993.txt', 'Defun1994.txt', 'Defun1995.txt', 'Defun1996.txt', 'Defun1997.txt', 'Defun1998.txt', 'Defun1999.txt', 'Defun2000.txt', 'Defun2001.txt', 'Defun2002.txt', 'Defun2003.txt', 'Defun2004.txt', 'Defun2005.txt', 'Defun2006.txt', 'Defun2007.txt', 'Defun2012.txt', 'Defun2013.txt']
✅ Defun1979.txt: 110400 filas, 14 columnas
✅ Defun1980.txt: 125573 filas, 14 columnas
✅ Defun1981.txt: 139505 filas, 14 columnas
✅ Defun1982.txt: 137678 filas, 14 columnas
✅ Defun1983.txt: 140292 filas, 14 columnas
✅ Defun1984.txt: 137189 filas, 14 columnas
✅ Defun1985.txt: 153947 filas, 14 columnas
✅ Defun1986.txt: 146346 filas, 14 columnas
✅ Defun1987.txt: 151957 filas, 14 columnas
✅ Defun1988.txt: 153067 filas, 14 columnas
✅ Defun1989.txt: 1

#### 2.1.1. Consistencia Campos

In [25]:
# Verificar que todas las columnas sean iguales en nombre y orden
nombres_columnas_base = None
columnas_consistentes = True

print("🔍 VERIFICACIÓN DE CONSISTENCIA DE COLUMNAS")
print("=" * 50)

for nombre_df, df in dataframes.items():
    columnas_actuales = df.columns.tolist()
    
    if nombres_columnas_base is None:
        # Establecer la primera como referencia
        nombres_columnas_base = columnas_actuales
        año_base = nombre_df.split('_')[1]
        print(f"📋 DataFrame base (defun_{año_base}): {nombres_columnas_base}")
        print("-" * 50)
    else:
        # Comparar con la base
        if columnas_actuales == nombres_columnas_base:
            print(f"✅ {nombre_df}: Columnas CONSISTENTES")
        else:
            print(f"❌ {nombre_df}: Columnas DIFERENTES")
            print(f"   Esperado: {nombres_columnas_base}")
            print(f"   Encontrado: {columnas_actuales}")
            columnas_consistentes = False

print("=" * 50)
if columnas_consistentes:
    print("🎉 TODAS las columnas son consistentes en nombre y orden")
else:
    print("⚠️  Hay inconsistencias en las columnas")

🔍 VERIFICACIÓN DE CONSISTENCIA DE COLUMNAS
📋 DataFrame base (defun_1979): ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
--------------------------------------------------
❌ defun_1980: Columnas DIFERENTES
   Esperado: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
   Encontrado: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', 'cons_exp', 'cau_homol']
❌ defun_1981: Columnas DIFERENTES
   Esperado: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
   Encontrado: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', '

#### 2.1.2. Estandarización Campos

In [26]:
# Estandarizar todas las columnas a MAYÚSCULAS (como 1979)
print("🔄 ESTANDARIZANDO COLUMNAS A MAYÚSCULAS")
print("=" * 40)

dataframes_estandarizados = {}

for nombre_df, df in dataframes.items():
    # Crear copia del DataFrame
    df_estandar = df.copy()
    
    # Convertir nombres de columnas a MAYÚSCULAS
    df_estandar.columns = [col.upper() for col in df_estandar.columns]
    
    dataframes_estandarizados[nombre_df] = df_estandar
    print(f"✅ {nombre_df}: {list(df.columns)} → {list(df_estandar.columns)}")

# Reemplazar el diccionario original
dataframes = dataframes_estandarizados

🔄 ESTANDARIZANDO COLUMNAS A MAYÚSCULAS
✅ defun_1979: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL'] → ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
✅ defun_1980: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', 'cons_exp', 'cau_homol'] → ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
✅ defun_1981: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', 'cons_exp', 'cau_homol'] → ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EX

In [28]:
# Acceder a defun_1979 y ver primeras 5 filas
defun_1979 = dataframes['defun_1979']

print("👀 PRIMERAS 5 FILAS DE DEFUN_1979")
print("=" * 50)
print(defun_1979.head())

👀 PRIMERAS 5 FILAS DE DEFUN_1979
  COD_DPTO COD_MUNIC A_DEFUN   ANO MES SEXO GRU_ED1 EST_CIVIL CODPTORE  \
0       50       001       1  1979  01    2      14         1       50   
1       50       223       1  1979  01    2      04         1       50   
2       50       223       1  1979  01    2      24         1       50   
3       50       683       1  1979  01    1      18         2       50   
4       50       287       1  1979  01    1      15         1       50   

  CODMUNRE SIT_DEFUN C_BAS1 CONS_EXP CAU_HOMOL  
0      001         1   6370        2       078  
1      223         2   2762        3       043  
2      223         2   2762        3       043  
3      683         2   8259        3       090  
4      287         2   9660        3       101  


#### 2.1.3. Verificar Consistencia 2

In [29]:
# Verificar consistencia después de estandarizar
print("\n🔍 VERIFICACIÓN POST-ESTANDARIZACIÓN")
print("=" * 50)

nombres_columnas_base = None
columnas_consistentes = True

for nombre_df, df in dataframes.items():
    columnas_actuales = df.columns.tolist()
    
    if nombres_columnas_base is None:
        nombres_columnas_base = columnas_actuales
        año_base = nombre_df.split('_')[1]
        print(f"📋 DataFrame base (defun_{año_base}): {nombres_columnas_base}")
        print("-" * 50)
    else:
        if columnas_actuales == nombres_columnas_base:
            print(f"✅ {nombre_df}: Columnas CONSISTENTES")
        else:
            print(f"❌ {nombre_df}: Columnas DIFERENTES")
            columnas_consistentes = False

print("=" * 50)
if columnas_consistentes:
    print("🎉 TODAS las columnas ahora son consistentes")
else:
    print("⚠️  Aún hay inconsistencias")


🔍 VERIFICACIÓN POST-ESTANDARIZACIÓN
📋 DataFrame base (defun_1979): ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
--------------------------------------------------
✅ defun_1980: Columnas CONSISTENTES
✅ defun_1981: Columnas CONSISTENTES
✅ defun_1982: Columnas CONSISTENTES
✅ defun_1983: Columnas CONSISTENTES
✅ defun_1984: Columnas CONSISTENTES
✅ defun_1985: Columnas CONSISTENTES
✅ defun_1986: Columnas CONSISTENTES
✅ defun_1987: Columnas CONSISTENTES
✅ defun_1988: Columnas CONSISTENTES
✅ defun_1989: Columnas CONSISTENTES
✅ defun_1990: Columnas CONSISTENTES
✅ defun_1991: Columnas CONSISTENTES
🎉 TODAS las columnas ahora son consistentes


#### 2.1.4. Unificar en un sólo DF defunciones 1979-1991

In [30]:
# Unificar todos los DataFrames en uno solo
print("🔄 UNIFICANDO DATAFRAMES (1979-1991)")
print("=" * 50)

# Verificar que todos los DataFrames están listos
print(f"📊 DataFrames a unificar: {len(dataframes)}")
for nombre, df in dataframes.items():
    print(f"   {nombre}: {df.shape}")

# Unificar todos los DataFrames
defun_1979_1991 = pd.concat(dataframes.values(), ignore_index=True)

print(f"\n✅ UNIFICACIÓN COMPLETADA")
print(f"📈 Tamaño del DataFrame unificado: {defun_1979_1991.shape}")
print(f"   - Filas: {len(defun_1979_1991):,}")
print(f"   - Columnas: {len(defun_1979_1991.columns)}")

🔄 UNIFICANDO DATAFRAMES (1979-1991)
📊 DataFrames a unificar: 13
   defun_1979: (110400, 14)
   defun_1980: (125573, 14)
   defun_1981: (139505, 14)
   defun_1982: (137678, 14)
   defun_1983: (140292, 14)
   defun_1984: (137189, 14)
   defun_1985: (153947, 14)
   defun_1986: (146346, 14)
   defun_1987: (151957, 14)
   defun_1988: (153067, 14)
   defun_1989: (154694, 14)
   defun_1990: (154685, 14)
   defun_1991: (163692, 14)

✅ UNIFICACIÓN COMPLETADA
📈 Tamaño del DataFrame unificado: (1869025, 14)
   - Filas: 1,869,025
   - Columnas: 14


In [53]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101


### 2.2. Leer Datos de Codigo Departamento y Codigo Municipio DANE

#### 2.2.1. Conectar archivo de Codigo Departamento y Codigo Municipio DANE

In [51]:
# Ruta del archivo DIVIPOLA de Centros Poblados
ruta_archivo = "data/raw/Referenciales/DIVIPOLA_CentrosPoblados.csv"

# Leer todo como texto para preservar formatos
try:
    divipola = pd.read_csv(
        ruta_archivo, 
        encoding='latin-1',
        sep=';',
        dtype=str  # Esto fuerza todo a texto
    )
    print(f"✅ Archivo leído correctamente (todo como texto)")
    print(f"   - Filas: {len(divipola):,}")
    print(f"   - Columnas: {len(divipola.columns)}")
    
    # Verificar que se mantienen los formatos
    print(f"\n📋 Primeras filas (Código_Departamento):")
    display(divipola.head(10))
    print("\n")
    
    print(f"\n🔍 Tipos de datos:")
    print(divipola.dtypes)
    
except Exception as e:
    print(f"❌ Error: {e}")

✅ Archivo leído correctamente (todo como texto)
   - Filas: 8,421
   - Columnas: 9

📋 Primeras filas (Código_Departamento):


,Código_Departamento,Nombre_Departamento,Código_Municipio,Nombre_Municipio,Código_Entidad,Nombre_Entidad,Tipo,Longitud,Latitud
0,05,ANTIOQUIA,05001,MEDELLÍN,05001000,"MEDELLÍN, DISTRITO ESPECIAL DE CIENCIA, TECNOL...",CM,"-75,581775","6,246631"
1,05,ANTIOQUIA,05001,MEDELLÍN,05001001,PALMITAS,CP,"-75,690573","6,343919"
2,05,ANTIOQUIA,05001,MEDELLÍN,05001004,SANTA ELENA,CP,"-75,501293","6,210599"
3,05,ANTIOQUIA,05001,MEDELLÍN,05001009,ALTAVISTA,CP,"-75,643706","6,221429"
4,05,ANTIOQUIA,05001,MEDELLÍN,05001010,AGUAS FRÍAS,CP,"-75,633948","6,233335"
5,05,ANTIOQUIA,05001,MEDELLÍN,05001013,SAN JOSÉ DEL MANZANILLO,CP,"-75,612136","6,206092"
6,05,ANTIOQUIA,05001,MEDELLÍN,05001014,BARRO BLANCO,CP,"-75,479856","6,236734"
7,05,ANTIOQUIA,05001,MEDELLÍN,05001015,EL CERRO,CP,"-75,489496","6,200314"
8,05,ANTIOQUIA,05001,MEDELLÍN,05001017,EL PATIO,CP,"-75,65111","6,277841"
9,05,ANTIOQUIA,05001,MEDELLÍN,05001018,EL PLACER,CP,"-75,492971","6,224788"





🔍 Tipos de datos:
Código_Departamento    object
Nombre_Departamento    object
Código_Municipio       object
Nombre_Municipio       object
Código_Entidad         object
Nombre_Entidad         object
Tipo                   object
Longitud               object
Latitud                object
dtype: object


### 2.3. Prepara para merge municipio y departamento

#### 2.3.1. Corregir códigos 83-18 (Caquetá)

In [54]:
print("\n1. 📝 CORRIGIENDO CÓDIGOS 83 → 18")
print("-" * 30)

defun_1979_1991_corregido = defun_1979_1991.copy()
defun_1979_1991_corregido.loc[defun_1979_1991_corregido['COD_DPTO'] == '83', 'COD_DPTO'] = '18'
print("✅ Códigos 83 → 18 actualizados")


1. 📝 CORRIGIENDO CÓDIGOS 83 → 18
------------------------------
✅ Códigos 83 → 18 actualizados


#### 2.3.2. Hacer merge (join en SQL)

In [55]:
print("\n2. 🔗 HACIENDO MERGE CON CÓDIGOS CORREGIDOS")
print("-" * 40)

defun_1979_1991_con_nombres = defun_1979_1991_corregido.merge(
    divipola_merge,
    on=['COD_DPTO', 'COD_MUNIC'],
    how='left'
)

print(f"✅ Merge completado")
print(f"📊 Dimensiones: {defun_1979_1991_con_nombres.shape}")

# Verificar registros sin match después de la corrección
sin_match = defun_1979_1991_con_nombres['NOMBRE_DEPARTAMENTO'].isna().sum()
print(f"🔍 Registros sin match: {sin_match:,} ({sin_match/len(defun_1979_1991_con_nombres)*100:.2f}%)")


2. 🔗 HACIENDO MERGE CON CÓDIGOS CORREGIDOS
----------------------------------------
✅ Merge completado
📊 Dimensiones: (1869025, 16)
🔍 Registros sin match: 4,271 (0.23%)


#### 2.3.3. Preparar cabeceras municipales

In [56]:
print("\n3. 🏛️ PREPARANDO CABECERAS MUNICIPALES")
print("-" * 35)

cabeceras_municipales = divipola_merge[
    divipola_merge['COD_MUNIC'] == '001'
][['COD_DPTO', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']].rename(
    columns={
        'NOMBRE_DEPARTAMENTO': 'NOMBRE_DEPTO_CABECERA',
        'NOMBRE_MUNICIPIO': 'NOMBRE_MUNIC_CABECERA'
    }
)

print(f"📋 Cabeceras encontradas: {len(cabeceras_municipales)}")



3. 🏛️ PREPARANDO CABECERAS MUNICIPALES
-----------------------------------
📋 Cabeceras encontradas: 33


#### 2.3.4. Completar registros sin match con cabeceras municipales

In [57]:
print("\n4. 🔄 COMPLETANDO REGISTROS SIN MATCH")
print("-" * 35)

sin_match_mask = defun_1979_1991_con_nombres['NOMBRE_DEPARTAMENTO'].isna()

defun_1979_1991_final = defun_1979_1991_con_nombres.merge(
    cabeceras_municipales,
    on='COD_DPTO',
    how='left'
)

defun_1979_1991_final.loc[sin_match_mask, 'NOMBRE_DEPARTAMENTO'] = defun_1979_1991_final.loc[sin_match_mask, 'NOMBRE_DEPTO_CABECERA']
defun_1979_1991_final.loc[sin_match_mask, 'NOMBRE_MUNICIPIO'] = defun_1979_1991_final.loc[sin_match_mask, 'NOMBRE_MUNIC_CABECERA']

defun_1979_1991_final = defun_1979_1991_final.drop(['NOMBRE_DEPTO_CABECERA', 'NOMBRE_MUNIC_CABECERA'], axis=1)

print("✅ Registros sin match completados")



4. 🔄 COMPLETANDO REGISTROS SIN MATCH
-----------------------------------
✅ Registros sin match completados


#### 2.3.5. Verificación Final

In [58]:
print("\n5. 📊 VERIFICACIÓN FINAL")
print("-" * 25)

sin_match_final = defun_1979_1991_final['NOMBRE_DEPARTAMENTO'].isna().sum()
print(f"🎯 Registros sin match finales: {sin_match_final:,}")

# Mostrar resultado
print(f"\n👀 PRIMERAS FILAS DEL RESULTADO FINAL:")
columnas_mostrar = ['COD_DPTO', 'COD_MUNIC', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO', 'ANO']
print(defun_1979_1991_final[columnas_mostrar].head(10))

# Ver códigos que aún no tienen match (si los hay)
if sin_match_final > 0:
    print(f"\n🔍 CÓDIGOS QUE AÚN NO TIENEN MATCH:")
    sin_match_df = defun_1979_1991_final[defun_1979_1991_final['NOMBRE_DEPARTAMENTO'].isna()]
    codigos_sin_match = sin_match_df[['COD_DPTO', 'COD_MUNIC']].drop_duplicates()
    print(codigos_sin_match)
    print(f"Total: {len(codigos_sin_match)} códigos únicos")

# Actualizar el DataFrame principal
defun_1979_1991 = defun_1979_1991_final.copy()
print("\n💾 DataFrame principal actualizado")


5. 📊 VERIFICACIÓN FINAL
-------------------------
🎯 Registros sin match finales: 0

👀 PRIMERAS FILAS DEL RESULTADO FINAL:
  COD_DPTO COD_MUNIC NOMBRE_DEPARTAMENTO   NOMBRE_MUNICIPIO   ANO
0       50       001                META      VILLAVICENCIO  1979
1       50       223                META           CUBARRAL  1979
2       50       223                META           CUBARRAL  1979
3       50       683                META  SAN JUAN DE ARAMA  1979
4       50       287                META      FUENTE DE ORO  1979
5       50       287                META      FUENTE DE ORO  1979
6       50       577                META      PUERTO LLERAS  1979
7       50       577                META      PUERTO LLERAS  1979
8       50       577                META      PUERTO LLERAS  1979
9       50       577                META      PUERTO LLERAS  1979

💾 DataFrame principal actualizado


In [59]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO


### 2.4. Otras homologaciones

#### 2.4.1. Transformar campo A_DEFUN

In [64]:
print("🏷️ CREANDO CAMPO A_DEFUN_DESC")
print("=" * 40)

# Crear el mapeo de valores
mapeo_a_defun = {
    "1": 'CABECERA MUNICIPAL',
    "2": 'RESTO', 
    "3": 'SIN INFORMACIÓN'
}

# Crear la nueva columna
defun_1979_1991['A_DEFUN_DESC'] = defun_1979_1991['A_DEFUN'].map(mapeo_a_defun)

print("✅ Campo A_DEFUN_DESC creado")

🏷️ CREANDO CAMPO A_DEFUN_DESC
✅ Campo A_DEFUN_DESC creado


In [65]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO,A_DEFUN_DESC
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO,CABECERA MUNICIPAL
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA,CABECERA MUNICIPAL
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO,CABECERA MUNICIPAL


#### 2.4.2. Transformar campo SEXO

In [66]:
print("🏷️ CREANDO CAMPO SEXO_DESC")
print("=" * 40)

# Crear el mapeo de valores
mapeo_a_defun = {
    "1": 'MASCULINO',
    "2": 'FEMENINO'
}

# Crear la nueva columna
defun_1979_1991['SEXO_DESC'] = defun_1979_1991['SEXO'].map(mapeo_a_defun)

print("✅ Campo SEXO_DESC creado")

🏷️ CREANDO CAMPO SEXO_DESC
✅ Campo SEXO_DESC creado


In [67]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO,A_DEFUN_DESC,SEXO_DESC
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO,CABECERA MUNICIPAL,FEMENINO
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA,CABECERA MUNICIPAL,MASCULINO
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO,CABECERA MUNICIPAL,MASCULINO


#### 2.4.3. Transformar campo GRU_ED1

In [71]:
print("🏷️ CREANDO CAMPO GRU_ED1_DESC")
print("=" * 45)

# Crear el mapeo de grupos de edad
mapeo_gru_ed1 = {
    '01': 'MENORES DE UN DÍA',
    '02': 'DE 1 A 6 DÍAS', 
    '03': 'DE 7 A 29 DÍAS',
    '04': 'DE 1 A 5 MESES',
    '05': 'DE 6 A 11 MESES',
    '06': 'DE UN AÑO',
    '07': 'DE 2 A 4 AÑOS',
    '08': 'DE 5 A 9 AÑOS',
    '09': 'DE 10 A 14 AÑOS',
    '10': 'DE 15 A 19 AÑOS',
    '11': 'DE 20 A 24 AÑOS',
    '12': 'DE 25 A 29 AÑOS',
    '13': 'DE 30 A 34 AÑOS',
    '14': 'DE 35 A 39 AÑOS',
    '15': 'DE 40 A 44 AÑOS',
    '16': 'DE 45 A 49 AÑOS',
    '17': 'DE 50 A 54 AÑOS',
    '18': 'DE 55 A 59 AÑOS',
    '19': 'DE 60 A 64 AÑOS',
    '20': 'DE 65 A 69 AÑOS',
    '21': 'DE 70 A 74 AÑOS',
    '22': 'DE 75 A 79 AÑOS',
    '23': 'DE 80 A 84 AÑOS',
    '24': 'DE 85 Y MÁS AÑOS',
    '25': 'EDAD DESCONOCIDA'
}

# Convertir GRU_ED1 a string (por si viene como número) y formatear a 2 dígitos
defun_1979_1991['GRU_ED1_STR'] = defun_1979_1991['GRU_ED1'].astype(str).str.zfill(2)

# Crear la nueva columna
defun_1979_1991['GRU_ED1_DESC'] = defun_1979_1991['GRU_ED1_STR'].map(mapeo_gru_ed1)

print("✅ Campo GRU_ED1_DESC creado")

🏷️ CREANDO CAMPO GRU_ED1_DESC
✅ Campo GRU_ED1_DESC creado


In [72]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO,A_DEFUN_DESC,SEXO_DESC,GRU_ED1_STR,GRU_ED1_DESC
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO,CABECERA MUNICIPAL,FEMENINO,14,DE 35 A 39 AÑOS
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,04,DE 1 A 5 MESES
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,24,DE 85 Y MÁS AÑOS
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA,CABECERA MUNICIPAL,MASCULINO,18,DE 55 A 59 AÑOS
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO,CABECERA MUNICIPAL,MASCULINO,15,DE 40 A 44 AÑOS


#### 2.4.4. Transformar campo EST_CIVIL

In [75]:
print("🏷️ CREANDO CAMPO EST_CIVIL_DESC")
print("=" * 50)

# Crear el mapeo de estados civiles en MAYÚSCULAS
mapeo_est_civil = {
    "1": 'SOLTERO',
    "2": 'CASADO', 
    "3": 'VIUDO',
    "4": 'EN UNIÓN LIBRE, DIVORCIADO Y OTRO',
    "5": 'SIN INFORMACIÓN'
}

# Crear la nueva columna
defun_1979_1991['EST_CIVIL_DESC'] = defun_1979_1991['EST_CIVIL'].map(mapeo_est_civil)

print("✅ Campo EST_CIVIL_DESC creado en MAYÚSCULAS")

🏷️ CREANDO CAMPO EST_CIVIL_DESC
✅ Campo EST_CIVIL_DESC creado en MAYÚSCULAS


In [78]:
pd.set_option('display.max_columns', None)
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO,A_DEFUN_DESC,SEXO_DESC,GRU_ED1_STR,GRU_ED1_DESC,EST_CIVIL_DESC
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO,CABECERA MUNICIPAL,FEMENINO,14,DE 35 A 39 AÑOS,SOLTERO
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,04,DE 1 A 5 MESES,SOLTERO
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,24,DE 85 Y MÁS AÑOS,SOLTERO
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA,CABECERA MUNICIPAL,MASCULINO,18,DE 55 A 59 AÑOS,CASADO
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO,CABECERA MUNICIPAL,MASCULINO,15,DE 40 A 44 AÑOS,SOLTERO


### 2.5. Homologar Municipio y Departamento de residencia

#### 2.5.1. Corregir codigos antiguos de departamentos

In [92]:
print("🔧 RECONSTRUYENDO EL PROCESO CORRECTAMENTE")
print("=" * 50)

# PASO 1: Volver al DataFrame original y hacer el merge correctamente
print("1. 🔄 HACIENDO MERGE CORRECTO PARA RESIDENCIA")

# Usar el DataFrame original (antes del merge problemático)
defun_1979_1991_original = defun_1979_1991.copy()

# Asegurar que los códigos sean strings y tengan el formato correcto
defun_1979_1991_original['CODPTORE'] = defun_1979_1991_original['CODPTORE'].astype(str).str.zfill(2)
defun_1979_1991_original['CODMUNRE'] = defun_1979_1991_original['CODMUNRE'].astype(str).str.zfill(3)

# Corregir código 83 → 18
defun_1979_1991_original.loc[defun_1979_1991_original['CODPTORE'] == '83', 'CODPTORE'] = '18'

# Hacer merge CORRECTO con suffixes
defun_1979_1991_correcto = defun_1979_1991_original.merge(
    divipola_merge,
    left_on=['CODPTORE', 'CODMUNRE'],
    right_on=['COD_DPTO', 'COD_MUNIC'],
    how='left',
    suffixes=('', '_RESIDENCIA')  # Usar suffixes para diferenciar
)

print("✅ Merge correcto completado")

# PASO 2: Renombrar las columnas de residencia correctamente
print("\n2. 🏷️ RENOMBRANDO COLUMNAS DE RESIDENCIA")

# Las columnas del merge estarán con el suffix _RESIDENCIA
# Buscar y renombrar las columnas correctas
columnas_actuales = defun_1979_1991_correcto.columns.tolist()
print("Columnas disponibles:", columnas_actuales)

# Renombrar manualmente si es necesario
if 'NOMBRE_DEPARTAMENTO_RESIDENCIA' in columnas_actuales:
    # Ya está correcto
    print("✅ Columnas ya tienen el nombre correcto")
else:
    # Buscar las columnas que necesitan renombre
    for col in columnas_actuales:
        if 'NOMBRE_DEPARTAMENTO' in col and col != 'NOMBRE_DEPARTAMENTO':
            defun_1979_1991_correcto = defun_1979_1991_correcto.rename(columns={col: 'NOMBRE_DEPARTAMENTO_RESIDENCIA'})
        elif 'NOMBRE_MUNICIPIO' in col and col != 'NOMBRE_MUNICIPIO':
            defun_1979_1991_correcto = defun_1979_1991_correcto.rename(columns={col: 'NOMBRE_MUNICIPIO_RESIDENCIA'})

print("✅ Columnas renombradas")

# PASO 3: Verificar el resultado
print("\n3. 📊 VERIFICANDO RESULTADO CORRECTO")

total = len(defun_1979_1991_correcto)

# Ahora deberían ser Series normales
if 'NOMBRE_DEPARTAMENTO_RESIDENCIA' in defun_1979_1991_correcto.columns:
    sin_depto = defun_1979_1991_correcto['NOMBRE_DEPARTAMENTO_RESIDENCIA'].isna().sum()
    sin_munic = defun_1979_1991_correcto['NOMBRE_MUNICIPIO_RESIDENCIA'].isna().sum()
    
    print(f"📊 RESULTADOS CORRECTOS:")
    print(f"Total registros: {total:,}")
    print(f"Sin departamento residencia: {sin_depto:,}")
    print(f"Sin municipio residencia: {sin_munic:,}")
    print(f"Porcentaje sin municipio: {(sin_munic/total*100):.2f}%")
    
    # Mostrar ejemplos
    print(f"\n👀 EJEMPLOS CORRECTOS:")
    columnas_ejemplo = ['COD_DPTO', 'NOMBRE_DEPARTAMENTO', 'CODPTORE', 'NOMBRE_DEPARTAMENTO_RESIDENCIA']
    print(defun_1979_1991_correcto[columnas_ejemplo].head(10))
else:
    print("❌ Las columnas de residencia no se crearon correctamente")
    print("Columnas disponibles:", defun_1979_1991_correcto.columns.tolist())

🔧 RECONSTRUYENDO EL PROCESO CORRECTAMENTE
1. 🔄 HACIENDO MERGE CORRECTO PARA RESIDENCIA
✅ Merge correcto completado

2. 🏷️ RENOMBRANDO COLUMNAS DE RESIDENCIA
Columnas disponibles: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO', 'A_DEFUN_DESC', 'SEXO_DESC', 'GRU_ED1_STR', 'GRU_ED1_DESC', 'EST_CIVIL_DESC', 'COD_DPTO_RESIDENCIA', 'COD_MUNIC_RESIDENCIA', 'NOMBRE_DEPARTAMENTO_RESIDENCIA', 'NOMBRE_MUNICIPIO_RESIDENCIA']
✅ Columnas ya tienen el nombre correcto
✅ Columnas renombradas

3. 📊 VERIFICANDO RESULTADO CORRECTO
📊 RESULTADOS CORRECTOS:
Total registros: 1,869,025
Sin departamento residencia: 50,334
Sin municipio residencia: 50,334
Porcentaje sin municipio: 2.69%

👀 EJEMPLOS CORRECTOS:
  COD_DPTO NOMBRE_DEPARTAMENTO CODPTORE NOMBRE_DEPARTAMENTO_RESIDENCIA
0       50                META       50                           META
1       50      

#### 2.5.2. Completar registros

In [93]:
print("\n4. 🏛️ COMPLETANDO REGISTROS SIN MATCH")
print("=" * 40)

if 'NOMBRE_DEPARTAMENTO_RESIDENCIA' in defun_1979_1991_correcto.columns:
    sin_match = defun_1979_1991_correcto['NOMBRE_DEPARTAMENTO_RESIDENCIA'].isna().sum()
    
    if sin_match > 0:
        print(f"🔍 Completando {sin_match:,} registros sin match...")
        
        # Crear cabeceras municipales
        cabeceras = divipola_merge[divipola_merge['COD_MUNIC'] == '001'][['COD_DPTO', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']]
        cabeceras = cabeceras.rename(columns={
            'NOMBRE_DEPARTAMENTO': 'DEPT_CAB', 
            'NOMBRE_MUNICIPIO': 'MUNIC_CAB'
        })
        
        # Merge con cabeceras
        defun_completo = defun_1979_1991_correcto.merge(
            cabeceras, 
            left_on='CODPTORE', 
            right_on='COD_DPTO', 
            how='left'
        )
        
        # Completar nulos
        mask = defun_completo['NOMBRE_DEPARTAMENTO_RESIDENCIA'].isna()
        defun_completo.loc[mask, 'NOMBRE_DEPARTAMENTO_RESIDENCIA'] = defun_completo.loc[mask, 'DEPT_CAB']
        defun_completo.loc[mask, 'NOMBRE_MUNICIPIO_RESIDENCIA'] = defun_completo.loc[mask, 'MUNIC_CAB']
        
        # Limpiar
        defun_completo = defun_completo.drop(['DEPT_CAB', 'MUNIC_CAB', 'COD_DPTO_y'], axis=1, errors='ignore')
        if 'COD_DPTO_x' in defun_completo.columns:
            defun_completo = defun_completo.rename(columns={'COD_DPTO_x': 'COD_DPTO'})
        
        defun_1979_1991 = defun_completo.copy()
        print("✅ Completado finalizado")
    else:
        defun_1979_1991 = defun_1979_1991_correcto.copy()
        print("✅ Todos los registros tienen match")
else:
    print("❌ No se pueden completar - columnas no disponibles")


4. 🏛️ COMPLETANDO REGISTROS SIN MATCH
🔍 Completando 50,334 registros sin match...
✅ Completado finalizado


#### 2.5.3. Verificar el resultado

In [95]:
print("🔍 VERIFICACIÓN COMPLETA DE CAMPOS DE RESIDENCIA")
print("=" * 55)

# 1. Verificar que las columnas existen y tienen el tipo correcto
print("1. 📋 EXISTENCIA Y TIPOS DE COLUMNAS")
print("-" * 35)

columnas_residencia = ['NOMBRE_DEPARTAMENTO_RESIDENCIA', 'NOMBRE_MUNICIPIO_RESIDENCIA']
columnas_defuncion = ['NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']

for col in columnas_residencia + columnas_defuncion:
    if col in defun_1979_1991.columns:
        dtype = defun_1979_1991[col].dtype
        print(f"✅ {col}: {dtype}")
    else:
        print(f"❌ {col}: NO EXISTE")

# 2. Verificar valores nulos
print(f"\n2. 📊 VALORES NULOS")
print("-" * 20)

total_registros = len(defun_1979_1991)
print(f"📈 Total registros: {total_registros:,}")

for col in columnas_residencia:
    if col in defun_1979_1991.columns:
        nulos = defun_1979_1991[col].isna().sum()
        porcentaje = (nulos / total_registros) * 100
        print(f"   {col}: {nulos:,} nulos ({porcentaje:.4f}%)")
        if nulos == 0:
            print(f"      🎉 PERFECTO - SIN NULOS!")

🔍 VERIFICACIÓN COMPLETA DE CAMPOS DE RESIDENCIA
1. 📋 EXISTENCIA Y TIPOS DE COLUMNAS
-----------------------------------
✅ NOMBRE_DEPARTAMENTO_RESIDENCIA: object
✅ NOMBRE_MUNICIPIO_RESIDENCIA: object
✅ NOMBRE_DEPARTAMENTO: object
✅ NOMBRE_MUNICIPIO: object

2. 📊 VALORES NULOS
--------------------
📈 Total registros: 1,869,025
   NOMBRE_DEPARTAMENTO_RESIDENCIA: 41,477 nulos (2.2192%)
   NOMBRE_MUNICIPIO_RESIDENCIA: 41,477 nulos (2.2192%)


#### 2.5.4. Revisar Casos Pendientes

In [96]:
print("🔧 COMPLETANDO REGISTROS NULOS RESTANTES")
print("=" * 45)

# Identificar los registros que aún tienen nulos
nulos_mask = defun_1979_1991['NOMBRE_DEPARTAMENTO_RESIDENCIA'].isna()
print(f"🔍 Registros con nulos: {nulos_mask.sum():,}")

# Ver qué códigos tienen problemas
if nulos_mask.sum() > 0:
    print(f"\n📋 CÓDIGOS PROBLEMÁTICOS:")
    codigos_problematicos = defun_1979_1991[nulos_mask][['CODPTORE', 'CODMUNRE']].drop_duplicates()
    print(codigos_problematicos)
    print(f"Total códigos problemáticos: {len(codigos_problematicos)}")

🔧 COMPLETANDO REGISTROS NULOS RESTANTES
🔍 Registros con nulos: 41,477

📋 CÓDIGOS PROBLEMÁTICOS:
        CODPTORE CODMUNRE
81            01      999
400625        75      998
420831        75      862
680741        75      591
722398        75      218
814555        75      076
814565        75      604
1746094       75      999
Total códigos problemáticos: 8


In [97]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO,A_DEFUN_DESC,SEXO_DESC,GRU_ED1_STR,GRU_ED1_DESC,EST_CIVIL_DESC,COD_DPTO_RESIDENCIA,COD_MUNIC_RESIDENCIA,NOMBRE_DEPARTAMENTO_RESIDENCIA,NOMBRE_MUNICIPIO_RESIDENCIA
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO,CABECERA MUNICIPAL,FEMENINO,14,DE 35 A 39 AÑOS,SOLTERO,50,001,META,VILLAVICENCIO
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,04,DE 1 A 5 MESES,SOLTERO,50,223,META,CUBARRAL
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,24,DE 85 Y MÁS AÑOS,SOLTERO,50,223,META,CUBARRAL
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA,CABECERA MUNICIPAL,MASCULINO,18,DE 55 A 59 AÑOS,CASADO,50,683,META,SAN JUAN DE ARAMA
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO,CABECERA MUNICIPAL,MASCULINO,15,DE 40 A 44 AÑOS,SOLTERO,50,287,META,FUENTE DE ORO


#### 2.5.5. Marcar nulos como NO REGISTRA

In [98]:
print("🏷️ MARCANDO NULOS COMO 'NO REGISTRA'")
print("=" * 40)

# Contar nulos antes del proceso
nulos_inicial_depto = defun_1979_1991['NOMBRE_DEPARTAMENTO_RESIDENCIA'].isna().sum()
nulos_inicial_munic = defun_1979_1991['NOMBRE_MUNICIPIO_RESIDENCIA'].isna().sum()

print(f"📊 NULOS INICIALES:")
print(f"   Departamento residencia: {nulos_inicial_depto:,}")
print(f"   Municipio residencia: {nulos_inicial_munic:,}")

# Marcar nulos como "NO REGISTRA"
defun_1979_1991.loc[defun_1979_1991['NOMBRE_DEPARTAMENTO_RESIDENCIA'].isna(), 'NOMBRE_DEPARTAMENTO_RESIDENCIA'] = 'NO REGISTRA'
defun_1979_1991.loc[defun_1979_1991['NOMBRE_MUNICIPIO_RESIDENCIA'].isna(), 'NOMBRE_MUNICIPIO_RESIDENCIA'] = 'NO REGISTRA'

print("✅ Nulos marcados como 'NO REGISTRA'")

🏷️ MARCANDO NULOS COMO 'NO REGISTRA'
📊 NULOS INICIALES:
   Departamento residencia: 41,477
   Municipio residencia: 41,477
✅ Nulos marcados como 'NO REGISTRA'


In [99]:
print("\n📊 VERIFICACIÓN FINAL DESPUÉS DE MARCADO")
print("=" * 45)

# Verificar que ya no hay nulos
nulos_final_depto = defun_1979_1991['NOMBRE_DEPARTAMENTO_RESIDENCIA'].isna().sum()
nulos_final_munic = defun_1979_1991['NOMBRE_MUNICIPIO_RESIDENCIA'].isna().sum()

print(f"🔍 NULOS FINALES:")
print(f"   NOMBRE_DEPARTAMENTO_RESIDENCIA: {nulos_final_depto:,}")
print(f"   NOMBRE_MUNICIPIO_RESIDENCIA: {nulos_final_munic:,}")

# Verificar distribución de valores
print(f"\n📈 DISTRIBUCIÓN DE VALORES:")
for col in ['NOMBRE_DEPARTAMENTO_RESIDENCIA', 'NOMBRE_MUNICIPIO_RESIDENCIA']:
    distribucion = defun_1979_1991[col].value_counts().head()
    print(f"\n📋 {col} - Top 5 valores:")
    for valor, count in distribucion.items():
        porcentaje = (count / len(defun_1979_1991)) * 100
        print(f"   {valor}: {count:,} ({porcentaje:.2f}%)")


📊 VERIFICACIÓN FINAL DESPUÉS DE MARCADO
🔍 NULOS FINALES:
   NOMBRE_DEPARTAMENTO_RESIDENCIA: 0
   NOMBRE_MUNICIPIO_RESIDENCIA: 0

📈 DISTRIBUCIÓN DE VALORES:

📋 NOMBRE_DEPARTAMENTO_RESIDENCIA - Top 5 valores:
   ANTIOQUIA: 322,781 (17.27%)
   BOGOTÁ, D.C.: 245,107 (13.11%)
   VALLE DEL CAUCA: 216,103 (11.56%)
   CUNDINAMARCA: 104,834 (5.61%)
   SANTANDER: 95,591 (5.11%)

📋 NOMBRE_MUNICIPIO_RESIDENCIA - Top 5 valores:
   BOGOTÁ, D.C.: 245,107 (13.11%)
   MEDELLÍN: 135,372 (7.24%)
   SANTIAGO DE CALI: 96,000 (5.14%)
   BARRANQUILLA: 59,015 (3.16%)
   NO REGISTRA: 41,477 (2.22%)


In [100]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO,A_DEFUN_DESC,SEXO_DESC,GRU_ED1_STR,GRU_ED1_DESC,EST_CIVIL_DESC,COD_DPTO_RESIDENCIA,COD_MUNIC_RESIDENCIA,NOMBRE_DEPARTAMENTO_RESIDENCIA,NOMBRE_MUNICIPIO_RESIDENCIA
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO,CABECERA MUNICIPAL,FEMENINO,14,DE 35 A 39 AÑOS,SOLTERO,50,001,META,VILLAVICENCIO
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,04,DE 1 A 5 MESES,SOLTERO,50,223,META,CUBARRAL
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,24,DE 85 Y MÁS AÑOS,SOLTERO,50,223,META,CUBARRAL
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA,CABECERA MUNICIPAL,MASCULINO,18,DE 55 A 59 AÑOS,CASADO,50,683,META,SAN JUAN DE ARAMA
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO,CABECERA MUNICIPAL,MASCULINO,15,DE 40 A 44 AÑOS,SOLTERO,50,287,META,FUENTE DE ORO


### 2.6. Transformaciones

#### 2.6.1. Transformar campo SIT_DEFUN_DESC

In [101]:
print("🏷️ CREANDO CAMPO SIT_DEFUN_DESC")
print("=" * 50)

# Crear el mapeo de estados civiles en MAYÚSCULAS
mapeo_est_civil = {
    "1": 'HOSPITAL O CLINICA',
    "2": 'CASA', 
    "3": 'OTRO SITIO',
    "4": 'SIN INFORMACIÓN'
}

# Crear la nueva columna
defun_1979_1991['SIT_DEFUN_DESC'] = defun_1979_1991['SIT_DEFUN'].map(mapeo_est_civil)

print("✅ Campo ESIT_DEFUN_DESC_DESC creado en MAYÚSCULAS")

🏷️ CREANDO CAMPO SIT_DEFUN_DESC
✅ Campo ESIT_DEFUN_DESC_DESC creado en MAYÚSCULAS


In [102]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO,A_DEFUN_DESC,SEXO_DESC,GRU_ED1_STR,GRU_ED1_DESC,EST_CIVIL_DESC,COD_DPTO_RESIDENCIA,COD_MUNIC_RESIDENCIA,NOMBRE_DEPARTAMENTO_RESIDENCIA,NOMBRE_MUNICIPIO_RESIDENCIA,SIT_DEFUN_DESC
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO,CABECERA MUNICIPAL,FEMENINO,14,DE 35 A 39 AÑOS,SOLTERO,50,001,META,VILLAVICENCIO,HOSPITAL O CLINICA
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,04,DE 1 A 5 MESES,SOLTERO,50,223,META,CUBARRAL,CASA
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,24,DE 85 Y MÁS AÑOS,SOLTERO,50,223,META,CUBARRAL,CASA
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA,CABECERA MUNICIPAL,MASCULINO,18,DE 55 A 59 AÑOS,CASADO,50,683,META,SAN JUAN DE ARAMA,CASA
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO,CABECERA MUNICIPAL,MASCULINO,15,DE 40 A 44 AÑOS,SOLTERO,50,287,META,FUENTE DE ORO,CASA


#### 2.6.2. Transformar campo C_BAS1

#### 2.6.2.1 Cargar la fuente de datos

In [105]:
# Ruta del archivo CIE (Clasificacin Internacional de Enfermedades)
ruta_archivo = "data/raw/Referenciales/CIE_9_10.xls"

# Leer todo como texto para preservar formatos
try:
    CIE = pd.read_excel(
        ruta_archivo
    )
    print(f"✅ Archivo leído correctamente (todo como texto)")
    print(f"   - Filas: {len(CIE):,}")
    print(f"   - Columnas: {len(CIE.columns)}")
    
    # Verificar que se mantienen los formatos
    print(f"\n📋 Primeras filas (Código_Departamento):")
    display(CIE.head(10))
    print("\n")
    
    print(f"\n🔍 Tipos de datos:")
    print(CIE.dtypes)
    
except Exception as e:
    print(f"❌ Error: {e}")

✅ Archivo leído correctamente (todo como texto)
   - Filas: 22,664
   - Columnas: 4

📋 Primeras filas (Código_Departamento):


,CIE9,LITERAL9,CIE10,LITERAL10
0,0010,COLERA DEBIDO A VIBRIO CHOLERAE,A000,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO CH..."
1,0010,COLERA DEBIDO A VIBRIO CHOLERAE,A009,"COLERA, NO ESPECIFICADO"
2,0011,COLERA DEBIDO A VIBRIO CHOLERAE EL TOR,A001,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO EL..."
3,0019,COLERA SIN ESPECIFICACION,A009,"COLERA, NO ESPECIFICADO"
4,0020,FIEBRE TIFOIDEA,A010,FIEBRE TIFOIDEA
5,0021,FIEBRE PARATIFOIDEA A,A011,FIEBRE PARATIFOIDEA A
6,0022,FIEBRE PARATIFOIDEA B,A012,FIEBRE PARATIFOIDEA B
7,0023,FIEBRE PARATIFOIDEA C,A013,FIEBRE PARATIFOIDEA C
8,0029,"FIEBRE PARATIFOIDEA, DE TIPO NO ESPECIFICADO",A014,"FIEBRE PARATIFOIDEA, NO ESPECIFICADA"
9,0030,GASTROENTERITIS DEBIDA A SALMONELLA,A020,ENTERITIS DEBIDA A SALMONELLA





🔍 Tipos de datos:
CIE9         object
LITERAL9     object
CIE10        object
LITERAL10    object
dtype: object


#### 2.6.2.2 Incorporar la información

In [116]:
print("=== REINICIANDO BÚSQUEDA C_BAS1_DESC ===")

# Paso 1: Eliminar la columna existente si existe
if 'C_BAS1_DESC' in defun_1979_1991.columns:
    defun_1979_1991 = defun_1979_1991.drop(columns=['C_BAS1_DESC'])
    print("✅ Columna C_BAS1_DESC eliminada")

# Paso 2: Realizar el cruce exacto inicial
print("\n=== CRUCE EXACTO INICIAL ===")
defun_1979_1991 = defun_1979_1991.merge(
    CIE[['CIE9', 'LITERAL9']],
    left_on='C_BAS1',
    right_on='CIE9',
    how='left'
)

# Renombrar la columna resultante
defun_1979_1991 = defun_1979_1991.rename(columns={'LITERAL9': 'C_BAS1_DESC'})

# Verificar el resultado inicial
coincidencias_iniciales = defun_1979_1991['C_BAS1_DESC'].notna().sum()
print(f"✅ Cruce exacto inicial completado")
print(f"📊 Coincidencias exactas: {coincidencias_iniciales:,} de {len(defun_1979_1991):,} ({coincidencias_iniciales/len(defun_1979_1991)*100:.2f}%)")

# Paso 3: Búsqueda priorizada para los que no coincidieron
sin_coincidencia_mask = defun_1979_1991['C_BAS1_DESC'].isna()
indices_sin_coincidencia = defun_1979_1991[sin_coincidencia_mask].index

print(f"\n=== BÚSQUEDA PRIORIZADA ===")
print(f"🔍 Registros pendientes por cruzar: {len(indices_sin_coincidencia):,}")

if len(indices_sin_coincidencia) > 0:
    
    def buscar_coincidencia_priorizada(c_bas1_val, cie_df):
        """
        Búsqueda priorizada por tipos de códigos CIE-9
        """
        if pd.isna(c_bas1_val):
            return None
        
        c_bas1_str = str(c_bas1_val).strip()
        
        # ESTRATEGIA 1: Búsqueda exacta directa (máxima prioridad)
        coincidencia_exacta = cie_df[cie_df['CIE9'] == c_bas1_str]
        if not coincidencia_exacta.empty:
            return coincidencia_exacta.iloc[0]['LITERAL9']
        
        # ESTRATEGIA 2: Códigos que empiezan con E (Causas externas - alta prioridad)
        if c_bas1_str.isdigit():
            variaciones_e = []
            
            # E + 4 dígitos completos (ej: 8910 -> E8910)
            if len(c_bas1_str) == 4:
                variaciones_e.append(f"E{c_bas1_str}")
            
            # E + primeros 3 dígitos (ej: 8910 -> E891)
            if len(c_bas1_str) >= 3:
                variaciones_e.append(f"E{c_bas1_str[:3]}")
            
            # E + sin ceros a la izquierda (ej: 0891 -> E891)
            sin_ceros = c_bas1_str.lstrip('0')
            if sin_ceros and sin_ceros != c_bas1_str:
                if len(sin_ceros) == 4:
                    variaciones_e.append(f"E{sin_ceros}")
                if len(sin_ceros) >= 3:
                    variaciones_e.append(f"E{sin_ceros[:3]}")
            
            # Probar todas las variaciones E
            for variacion in variaciones_e:
                coincidencia = cie_df[cie_df['CIE9'] == variacion]
                if not coincidencia.empty:
                    return coincidencia.iloc[0]['LITERAL9']
        
        # ESTRATEGIA 3: Códigos que empiezan con V (Contactos con servicios de salud)
        if c_bas1_str.isdigit():
            variaciones_v = []
            
            # V + 4 dígitos completos
            if len(c_bas1_str) == 4:
                variaciones_v.append(f"V{c_bas1_str}")
            
            # V + primeros 3 dígitos
            if len(c_bas1_str) >= 3:
                variaciones_v.append(f"V{c_bas1_str[:3]}")
            
            # V + sin ceros a la izquierda
            sin_ceros = c_bas1_str.lstrip('0')
            if sin_ceros and sin_ceros != c_bas1_str:
                if len(sin_ceros) == 4:
                    variaciones_v.append(f"V{sin_ceros}")
                if len(sin_ceros) >= 3:
                    variaciones_v.append(f"V{sin_ceros[:3]}")
            
            # Probar variaciones V
            for variacion in variaciones_v:
                coincidencia = cie_df[cie_df['CIE9'] == variacion]
                if not coincidencia.empty:
                    return coincidencia.iloc[0]['LITERAL9']
        
        # ESTRATEGIA 4: Otros prefijos comunes
        otros_prefijos = ['M', 'N']
        if c_bas1_str.isdigit():
            for prefijo in otros_prefijos:
                if len(c_bas1_str) >= 3:
                    # Prefijo + primeros 3 dígitos
                    codigo = f"{prefijo}{c_bas1_str[:3]}"
                    coincidencia = cie_df[cie_df['CIE9'] == codigo]
                    if not coincidencia.empty:
                        return coincidencia.iloc[0]['LITERAL9']
                
                # Prefijo + sin ceros a la izquierda
                sin_ceros = c_bas1_str.lstrip('0')
                if sin_ceros and len(sin_ceros) >= 3:
                    codigo = f"{prefijo}{sin_ceros[:3]}"
                    coincidencia = cie_df[cie_df['CIE9'] == codigo]
                    if not coincidencia.empty:
                        return coincidencia.iloc[0]['LITERAL9']
        
        # ESTRATEGIA 5: Búsqueda por contenido
        coincidencias_contenido = cie_df[
            cie_df['CIE9'].astype(str).str.contains(c_bas1_str, na=False)
        ]
        if not coincidencias_contenido.empty:
            return coincidencias_contenido.iloc[0]['LITERAL9']
        
        return None

    # Aplicar la búsqueda priorizada
    nuevos_valores = {}
    
    print("🔍 Ejecutando búsqueda priorizada...")
    print("   Prioridad: Exacto → E → V → M/N → Contenido")
    
    for idx in tqdm(indices_sin_coincidencia, desc="Búsqueda priorizada"):
        c_bas1_val = defun_1979_1991.loc[idx, 'C_BAS1']
        nuevo_valor = buscar_coincidencia_priorizada(c_bas1_val, CIE)
        if nuevo_valor:
            nuevos_valores[idx] = nuevo_valor

    # Actualizar el DataFrame
    for idx, valor in nuevos_valores.items():
        defun_1979_1991.at[idx, 'C_BAS1_DESC'] = valor

    print(f"✅ Nuevas coincidencias encontradas: {len(nuevos_valores):,}")

# Paso 4: Búsqueda manual para casos específicos conocidos
print(f"\n=== BÚSQUEDA MANUAL PARA CASOS ESPECÍFICOS ===")

# Casos específicos que sabemos que existen
casos_especificos = {
    '8910': 'E8910',  # Incendio en edificaciones
    '0891': 'E891',   # Versión con cero inicial
    '0893': 'E893',   # Versión con cero inicial
    '8930': 'E8930',  # Otros humos y gases
    '8560': 'E856',   # Accidente por caída
    '8570': 'E857',   # Accidente de vehículo
    '5840': 'E584',   # Exposición a factores naturales
    '8990': 'E899',   # Otros accidentes
}

actualizaciones_manuales = 0
for valor_original, codigo_cie in casos_especificos.items():
    # Verificar si este valor existe en nuestros datos sin descripción
    registros_pendientes = defun_1979_1991[
        (defun_1979_1991['C_BAS1'] == valor_original) & 
        (defun_1979_1991['C_BAS1_DESC'].isna())
    ]
    
    if len(registros_pendientes) > 0:
        # Buscar en CIE
        coincidencia_cie = CIE[CIE['CIE9'] == codigo_cie]
        
        if not coincidencia_cie.empty:
            descripcion = coincidencia_cie.iloc[0]['LITERAL9']
            print(f"✅ Actualizando manual: {valor_original} -> {codigo_cie}")
            
            # Actualizar los registros
            for idx in registros_pendientes.index:
                defun_1979_1991.at[idx, 'C_BAS1_DESC'] = descripcion
                actualizaciones_manuales += 1

print(f"✅ Actualizaciones manuales: {actualizaciones_manuales:,}")

# Paso 5: Limpiar columna temporal y mostrar resultados finales
print(f"\n=== LIMPIEZA Y RESULTADOS FINALES ===")

# Eliminar columna temporal CIE9 si existe
if 'CIE9' in defun_1979_1991.columns:
    defun_1979_1991 = defun_1979_1991.drop(columns=['CIE9'])
    print("✅ Columna temporal 'CIE9' eliminada")

# Resultados finales
coincidencias_finales = defun_1979_1991['C_BAS1_DESC'].notna().sum()
porcentaje_final = (coincidencias_finales / len(defun_1979_1991)) * 100
sin_coincidencia_final = defun_1979_1991['C_BAS1_DESC'].isna().sum()

print(f"\n🎯 RESULTADOS FINALES:")
print(f"📊 Coincidencias totales: {coincidencias_finales:,} de {len(defun_1979_1991):,} ({porcentaje_final:.2f}%)")
print(f"📈 Mejora total: {coincidencias_finales - coincidencias_iniciales:,} nuevas coincidencias")
print(f"❌ Registros sin coincidencia: {sin_coincidencia_final:,}")

# Mostrar ejemplos de actualizaciones exitosas
actualizaciones_exitosas = defun_1979_1991[
    (defun_1979_1991['C_BAS1_DESC'].notna()) & 
    (defun_1979_1991.index.isin(indices_sin_coincidencia))
]

if len(actualizaciones_exitosas) > 0:
    print(f"\n📋 EJEMPLOS DE ACTUALIZACIONES EXITOSAS:")
    display(actualizaciones_exitosas[['C_BAS1', 'C_BAS1_DESC']].head(10))

# Análisis de valores aún sin coincidencia
if sin_coincidencia_final > 0:
    valores_sin_coincidencia = defun_1979_1991.loc[
        defun_1979_1991['C_BAS1_DESC'].isna(), 'C_BAS1'
    ].unique()
    
    print(f"\n🔍 VALORES ÚNICOS SIN COINCIDENCIA ({len(valores_sin_coincidencia)}):")
    print(f"   {valores_sin_coincidencia}")

print(f"\n✅ PROCESO COMPLETADO EXITOSAMENTE!")

=== REINICIANDO BÚSQUEDA C_BAS1_DESC ===
✅ Columna C_BAS1_DESC eliminada

=== CRUCE EXACTO INICIAL ===
✅ Cruce exacto inicial completado
📊 Coincidencias exactas: 22,640,083 de 23,155,078 (97.78%)

=== BÚSQUEDA PRIORIZADA ===
🔍 Registros pendientes por cruzar: 514,995
🔍 Ejecutando búsqueda priorizada...
   Prioridad: Exacto → E → V → M/N → Contenido


Búsqueda priorizada:  10%|█         | 53229/514995 [09:25<1:21:48, 94.07it/s] 


KeyboardInterrupt: 

In [117]:
print(f"El DataFrame tiene {len(defun_1979_1991):,} filas")

El DataFrame tiene 23,155,078 filas
